In [1]:
import os, sys
import pandas as pd
from sklearn.manifold import MDS
from sklearn.cluster import KMeans

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from techminer2 import *

# import_scopus_file(scopus_file='raw-documents.csv',)
# import_references(scopus_file="raw-references.csv")

directory = "/workspaces/techminer2/data/"
import_scopus_file(directory)

- INFO - 248 raw records found in /workspaces/techminer2/data/raw-documents.csv.
- INFO - Main abstract texts saved to /workspaces/techminer2/data/abstracts.csv
- INFO - Computing Bradford Law Zones ...
- INFO - Searching local references using DOI ...


100%|██████████| 248/248 [00:00<00:00, 386.93it/s]


- INFO - Searching local references using document titles ...


100%|██████████| 248/248 [00:00<00:00, 392.32it/s]


- INFO - Consolidating local references ...
- INFO - Computing local citations ...
- INFO - Computing Bradford Law Zones ...
- INFO - Documents saved/merged to '/workspaces/techminer2/data/documents.csv'
- INFO - Post-processing docuemnts ...
- INFO - Creating institutions thesaurus ...
- INFO - Affiliations without country detected - check file /workspaces/techminer2/data/ignored_affiliations.txt
- INFO - Affiliations without country detected - check file /workspaces/techminer2/data/ignored_affiliations.txt
- INFO - Thesaurus file '/workspaces/techminer2/data/institutions.txt' created.
- INFO - Creating keywords thesaurus ...
- INFO - Thesaurus file '/workspaces/techminer2/data/keywords.txt' created.
- INFO - Applying thesaurus to institutions ...
- INFO - Extract and cleaning institutions.
- INFO - Extracting institution of first author ...
- INFO - The thesaurus was applied to institutions.
- INFO - Applying thesaurus to 'raw_author_keywords' column ...
- INFO - Applying thesaurus t

In [ ]:
import pandas as pd

documents = pd.read_csv("documents.csv")
documents.raw_keywords


In [ ]:
documents.raw_index_keywords

In [ ]:
documents.raw_author_keywords

In [ ]:
from techminer import *
directory = "/workspaces/techminer-api/data/"
collaboration_indicators("countries", directory=directory).head()

In [ ]:
import pandas as pd

m = pd.DataFrame(
    {
    'a': [1, 1, 1, 2, 2, 2],
    'b': ['A', 'B', 'D', 'A', 'B', 'C'],
    'v': [1, 2, 3, 4, 5, 6]
    }
)
m.pivot(index='a', columns='b', values='v')

In [ ]:
import pandas as pd

sorted(pd.read_csv("documents.csv").columns)